In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import HTML
from jupyter_plotly_dash import JupyterDash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
import numpy as np
import os
from sys import path as syspath
syspath.append(os.path.expanduser("~/srdjan_functs/"))
from sys import exc_info
from caiman import movie as cmovie
from caiman import load as cload
import json

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 1
%aimport Regions, physio_def_1, numeric, Automatic

from physio_def_1 import *
from Regions import Regions
from numeric import *
from Automatic import *

In [3]:
os.chdir(os.path.expanduser("~"))

In [54]:
# exampleNpzFile = "local_data/testdir/Experiment38a_7_Series016.npz"
exampleNpzFile = "local_data/Sandra/2020_03_06_Cosmin/Untitled_0_TEA10mM_20Hz_25ms_exposure_(30_sec)/TEA10mM_20Hz_25ms_exposure_(30_sec).tif"
# exampleNpzFile = "local_data/MB/Marjan_SOC2/20160218/Experiment_10_Series022.npz"
# exampleNpzFile = "local_data/Sandra/2019_10_16/Experiment39a/Experiment39a_13_Series024.npz"
# exampleNpzFile = "local_data/Sandra/2019_10_16/Experiment39a/Experiment39a_8_Series019.npz"

In [55]:
outputStyle = {
    "color":"blue",
    "font-family":"Courier New",
    "font-size":"80%",
    "max-width": "700px",
    }
infoStyle = {
    "font-size":"70%",
    "font-family":"Arial",
    # "background-color":"lightcyan",
    "color":"grey",
    "max-width": "700px",
    }
bodyStyle = {
    "font-family":"Arial",
    "max-width": "700px",
    }

In [90]:
def saveRois(regions,outDir,filename="",movie=None,cols=["trace"],):
    feedback = []
    try:
        from copy import deepcopy
        from datetime import date
        import pickle
        import pandas as pd
        from os.path import isdir
        from os import makedirs
        regions.sortFromCenter()
        if movie is not None:
            regions.update(movie)
        filename = filename.replace(" ","_")
        today = date.today()
        if len(filename):
            filename = "_".join([today.strftime("%Y_%m_%d"),filename])
        else:
            filename = today.strftime("%Y_%m_%d")
        if not isdir(outDir):
            makedirs(outDir)
            feedback += f"Output {outdir} directory created."

        traces = pd.DataFrame(np.vstack(regions.df.trace).T)
        traces["time"] = traces.index/regions.movie.fr
        traces = traces[["time"]+list(traces.columns[:-1])]
        tracefile = f"{outDir}/{filename}_trace.csv"
        traces.to_csv(tracefile, index=False)

        feedback += [f"Traces saved under {tracefile}.",html.Br()]

        saving = ['statImages', 'mode', 'image', 'filterSize', 'df']
        allAttrs = list(regions.__dict__.keys())
        subRegions = deepcopy(regions)
        for k in allAttrs:
            if k not in saving:
                del subRegions.__dict__[k]
        for k in regions.df.columns:
            if k not in ["peak","pixels"]:
                del subRegions.df[k]
        roifile = f"{outDir}/{filename}_rois.pkl"
        with open(roifile,"wb") as f:
            pickle.dump(subRegions,f)
        feedback += [f"ROI info saved under {roifile}."]
    except:
        from sys import exc_info
        feedback += ["ERROR: "+ exc_info().__repr__()]
    return feedback

In [91]:
baseFig = getFigure(w=200,h=200)

In [92]:
## global vars
test = False#"srdjan" in os.getcwd()
# movieFilename = None
# metadata = None
# movie = None
regions = None
narrowHover = False#bool(test)
mode = "diff_std"
debug = False

In [93]:
app = JupyterDash(__name__,
                  width=1400,
#                   height=1400,
                 )
APP_LAYOUT = [
    html.Div("Enter path to tiff file.",style=bodyStyle),
    html.Div(dcc.Input(id="filename_input",
            type="text",
            placeholder=exampleNpzFile,
            debounce=True,
            size=40,
            value=exampleNpzFile if test else "",
        ),style={**bodyStyle, "display":"inline-block"}),
    html.Div(id="filename_intermed",children="",style={**bodyStyle,"display":"inline-block"}),
    
    html.Div(dcc.Input(id="filename_hidden_input",
            type="text",
            debounce=False,
            size=1,
            value="",
        ),style={"display":"inline-block"}),
    html.Div(id="filename_output",children="Waiting for input",style={**bodyStyle,}),
    html.Video(id="video",
               src='local_data/Sandra/2020_03_06_Cosmin/Untitled_0_TEA10mM_20Hz_25ms_exposure_(30_sec)/TEA10mM_20Hz_25ms_exposure_(30_sec).mp4',               
               width=10,
               height=10,
              ),
    html.Div(dcc.Input(id="mode_image_data",
            type="text",
            debounce=False,
            size=10 if test else 1, 
            value="",
        ),
#              style={"display":"inline-block"}
            ),
    html.Div([
        html.Div("Use the slider below to set the threshold that eliminates the background.", style=bodyStyle),
        dcc.Graph(id='mode_image_histogram',figure=baseFig,config={"displayModeBar": False}),
        html.Div(dcc.Slider(id="mode_image_threshold_slider",
                            min=-2.1,max=0.1,value=-1,
                            marks = dict(zip([-2,-1,0],[0.01,.1,1])),
                            step=.01
                           ), style={"width":"400px"}),
        html.Div(id="slider_feedback",children="Waiting for input",style={**bodyStyle,}),
    ],style={"display":"inline-block"}),
    
    html.Div([
        dcc.Graph(id='stat_figure',figure=baseFig,config={"displayModeBar": False}),
    ],style={"display":"inline-block"}),
    
    html.Br(),
    html.Div(children=["Spatial filtering "],style={**bodyStyle,'display': 'inline-block'}),
    dcc.Input(id="gSig_filt_input",
        type="text",
        debounce=True,
        size = 5,
        value="",
        placeholder="e.g. 15",
        style={'display': 'inline-block'}),    

    html.Div(html.Abbr(
        children="[hover mouse for more info]",
        title="""For the proper analysis, you need to set the size of the spatial filter. Its value should be of the order of the  half of the typical cell dimension (in pixels!). """#In many of our recording, pixel size is around 2 µm. If we consider a typical cell of 10 µm, filter should be around 10/2/2 = 2.5. It needs to be an integer, so reasonable numbers to put would be 2 or 3, perhaps even 4. You can change it later.
        ),style={**infoStyle,'display': 'inline-block'}),

    html.Div(id="roi_feedback", children="",style=outputStyle),
    
#     dcc.Graph(id='stat_images',figure=baseFig,),
    html.Div([
        "Show downsampled to",
        dcc.Input(id="show_freq_input",
            type="text",
            debounce=False,
            size = 3,
            value='5'),
        "Hz"
        ], style={**bodyStyle,"display":"inline-block", "width":"49%"}, ),
    
    html.Br(),
    html.Div(dcc.Graph(id='roi_selector',    figure=baseFig), style={"display":"inline-block"}),
    html.Div(dcc.Graph(id='show_hover_trace',figure=baseFig), style={"display":"inline-block"}),
    html.Br(),
    html.Div(children=
        html.Pre(
        id="hover_out",
        children="hover",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px" if debug else "0px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
            "display"  : "inline-box"
              }),
             style={"display":"inline-block"}),
    html.Div(children=
        html.Pre(
        id="selected_out",
        children="selected",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px" if debug else "0px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
            "display"  : "inline-box"
              }),
             style={"display":"inline-block"}),
    html.Div(children=
        html.Pre(
        id="hover_plot_out",
        children="hover plot out",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px" if debug else "0px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
            "display"  : "inline-box"
              }),
             style={"display":"inline-block"}),
    html.Div(children=
        html.Pre(
        id="spare_out",
        children="spare",
        style={
            "vertical-align":"top",
            "width"    : "200px",
            "height"   : "100px" if debug else "0px",
            'border'   : 'thin lightgrey solid',
            'overflowY': 'scroll',
            "display"  : "inline-box"
              }),
             style={"display":"inline-block"}),
    
    html.Div([
       "Trace ROIs:",
        dcc.Input(id="selected_indices",
            type="text",
            debounce=True,
            size=60,
            value="",
         ), 
    ],style={**bodyStyle,"display":"inline-box"}),
    
#     html.Div([
#         html.Div("Show with offsets", style={**bodyStyle,"display":"inline-block"}),
#         html.Div(dcc.Input(id="offset_input",
#             type="text",
#             debounce=False,
#             size=3,
#             value='1'), style={**bodyStyle,"display":"inline-block"}),
#         html.Div("(only when showing filtered)", style={**infoStyle,"display":"inline-block"}),
#     ]),
#     html.Br(),
    
    html.Div(dcc.Graph(id='show_selected_traces',figure=getFigure(w=700))),
    
    html.H3("Save all"),
    html.Div([
        "When you are happy with your analysis, you can save it. By default, the analysis is saved as the current date (e.g. 2020_02_18) in the directory",
        html.Div("",style=outputStyle,id="analysis_folder"),
        "You can modify the filename by adding to it some other identifier(s) by entering them in the input box below. I suggest your username.",
        "When you are ready, press save."
    ]),
    dcc.Input(id="save_file_append",
        type="text",
        debounce=False,
        size = 30,
        placeholder="e.g. username",
        value='',style={"display":"inline-box"}),
    html.Button('Save', id='save_button', style={"display":"inline-box"},
                n_clicks=0
               ),
    html.Div(id="save_feedback",
             style={"display":"inline-box",**outputStyle}
            )
    ]

In [94]:
@app.callback(
    [Output("filename_intermed"     , "children" ),
     Output("filename_hidden_input" , "value"    ),
    ],
    [Input("filename_input", "value")],
    )
def filename_IO(input_text):
    if len(input_text):
        return "Loading...",input_text

In [95]:
@app.callback(
    [Output("filename_output", "children"),
     Output("video",           "src"),
     Output("mode_image_data", "value"),
#      Output("stat_figure", "figure"),
    ],
    [Input("filename_hidden_input", "value")],
    )
def filename_IO1(input_text):
    if len(input_text):
        global regions
        try:
            movie = cload(input_text)
            movieFilename = input_text
            out = f"{input_text} loaded successfully. "
        except:
            out = "Could not load %s. "%input_text
        try:
            freq = float(movieFilename.split("Hz")[-2].split("_")[-1])
            out += "Frequency set to %g Hz. "%freq
        except:
            freq = 1
            out += "Could not recognize frequency. Setting to %g Hz. "%freq
        movie.fr = freq
        try:
            if regions is not None: assert False
#             statImages = getStatImages(movie)
#             regions = Regions(statImages, mode=mode, diag=np.mean(statImages["mean"].shape)<400)
            regions = Regions(movie, mode=mode,full=False)
            regions.movie = movie
            regions.movieFilename = movieFilename
            out += "Stat images created. "
        except: pass
        
        try:
            im = regions.statImages[regions.mode]
            x = im.flatten()
            x[x<=0] = x[x>0].min()
            x = np.log10(x)
            h,be = np.histogram(x,100)
            x = (be[:-1]+be[1:])/2
            histData = json.dumps({"x":list(x),"y":list(h.astype("float"))})
            ff = go.Figure()
            ff.add_heatmap(z=im, hoverinfo='skip',showscale=False,colorscale=plxcolors.sequential.Jet)
            ff.update_layout({"width":500})
            ff.show(config={"displayModeBar": False})
            if debug: out += "Hist passed. "
        except:
            if debug: out += "Hist data could not be made. "+exc_info().__repr__()
            histData = json.dumps({"x":[],"y":[]})
        return out, ".".join(movieFilename.split(".")[:-1]+["mp4"]), histData

In [96]:
@app.callback(
    [Output("mode_image_histogram", "figure"),
     Output("stat_figure", "figure"),
     Output("slider_feedback", "children"),
    ],
    [Input("mode_image_data", "value"),
     Input("mode_image_threshold_slider", "value"),],
    )
def image_hist(input_text, thval):
    try:
        if len(input_text):
            data = eval(input_text)
            x = 10**np.array(data["x"])
            y = np.array(data["y"])
            thval = max(10**thval,x[np.where(np.cumsum(y)/y.sum()>.010)[0][0]])
            histFig = go.Figure(data=[
#                 go.Trace(x=x,y=y, hoverinfo="skip", showlegend=False, fill="tozeroy"),
                go.Bar(x=x,y=y, hoverinfo="skip", showlegend=False, marker={"color":np.log10(x),"colorscale":plxcolors.sequential.Jet},width=x/len(x)*3),
                go.Trace(x=[thval,thval],y=[-y.max()*2,y.max()*2], hoverinfo="skip", showlegend=False,marker={"color":"black"}),
                                     ])
            im = regions.statImages[regions.mode].copy()
            im[im<=0] = im[im>0].min()
            im[im<thval] = im.min()
            im = np.log10(im)
            modeFig = go.Figure()
            modeFig.add_heatmap(z=im, hoverinfo='skip',showscale=False,colorscale=plxcolors.sequential.Jet)
        else:
            histFig = baseFig()
            modeFig = baseFig() 
        histFig.update_layout(dict(
            width = 500,
            height = 380,
            xaxis_type="log",
            margin = dict(l=10, r=10, t=50, b=10),
            yaxis = {"range":[0,y.max()*1.07]}

        ))
        modeFig.update_layout({
            "width":400,
            "height":440,
            "margin":dict(l=10, r=10, t=50, b=10),})
        

        return histFig, modeFig, "th=%g"%thval
    except:
        return baseFig(), baseFig(), exc_info().__repr__()

In [97]:
@app.callback(
    [Output("roi_feedback", "children"),
#      Output("stat_images", "figure"),
     Output("roi_selector", "figure"),
#      Output("roi_choice","value"),
    ],
    [Input("gSig_filt_input", "value")],
    [State("slider_feedback", "children"),]
)
def calcRois(val,thtxt):
    if val == "":
        return ("",baseFig)
    try:
        global movie,regions
        fltrSize = eval(val)
        if regions is not None:
            if hasattr(regions, "fltrSize"):
                if regions.fltrSize==fltrSize:
                    return ("",baseFig())
        if type(fltrSize) == int:
            fltrSize = [fltrSize]
        cell_half_width = fltrSize[0]
        th = float(thtxt.split("=")[-1])
#         regions = Regions(regions.statImages,fltrSize, diag=True, mode=mode, )
        im = regions.statImages[regions.mode]
        regions.constructRois(im,img_th=th, diag=np.mean(im.shape)<400, gSig_filt=fltrSize)
        regions.purge_lones((fltrSize[0]/4)**2)
        regions.sortFromCenter()
        regions.calcTraces()
#         regions = createRegions(regions.statImages, diag=True, gSig_filt=fltrSize, drop_weak=False, mode=mode, )
        feedback = [f"Size of the spatial filter set to {fltrSize}.",html.Br()]
        feedback += [f" Number of detected ROIs: {len(regions.df)}"]
#         roisImage = showRoisOnly(regions,indices=range(len(regions.df)),im=regions.statImages["diff_std"])
#         if narrowHover:
#             roisImage.update_layout({"width":220,"height":240})
#         statsFig =  plotStatsFig(regions, showRois=True)
        roisImage = showRoisOnly(regions,indices=range(len(regions.df)),im=regions.statImages[mode])
        return (feedback,roisImage, )
    except:
        return ("ERROR from recalcRois: "+exc_info().__repr__(),baseFig)

In [98]:
def parseCol(regions, showFreq, filtered):
    if not hasattr(regions,"time"):
        regions.calcTraces(movie)
    if "trace" not in regions.df:
        regions.calcTraces(movie)
    if filtered:
        col = "showFilteredTrace_%.1f"%showFreq
        col0 = "filteredTrace"
    else:
        col = "showTrace_%.1f"%showFreq
        col0 = "trace"
    regions.df[col0]
    if not hasattr(regions,"showTime"):
        print ('creating empty showTime.')
        regions.showTime = {}
    if col not in regions.df.columns:
        n_Avg = int(regions.movie.fr/showFreq)
        n_Avg = max(1,n_Avg)
        if n_Avg<=1:
            regions.showTime[col] = regions.time
            regions.df[col] = regions.df[col0]
        else:
            print ('creating',col)
            regions.showTime[col] = rebin(regions.time,n_Avg)
            regions.df[col] = list(np.ones((len(regions.df),len(regions.showTime[col]))))
            #if filtered:
                #regions.df[col+"_slow"] = list(np.ones((len(regions.df),len(regions.showTime[col]))))
            for i in regions.df.index:
                if i==0:
                    print ("filling in", col)
                regions.df[col].loc[i] = rebin(regions.df.loc[i,col0],n_Avg)
                #regions.df[col+"_slow"].loc[i] = rebin(regions.df.loc[i,col0+"_slow"],n_Avg)
    return col

In [99]:
@app.callback(
    [Output("show_hover_trace", "figure"),
     Output("hover_plot_out", "children")],
    [Input("roi_selector", "hoverData")],
    [State("show_freq_input", "value"),
#      State("show_filtered_choice", "value"),
    ])
def plotHovered_callback(hoverData, showFreq, showFiltered=[]):
    try:
        global regions
        showFreq = float(showFreq)
        filtered = bool(len(showFiltered))
        colToshow = parseCol(regions, showFreq, filtered)
        showCol = [colToshow]
        #if filtered:
            #try: showCol += [colToshow+"_slow"]
            #except: pass
        try:
            hoverData.dtype
            ix = hoverData
        except:
            ix = [int(hoverData["points"][0]["hovertext"])]

#         trfig = plotTraces(regions,indices=ix,time=regions.showTime[colToshow], showCol=showCol)
        trfig = plotTraces(regions,indices=ix,time=regions.showTime[colToshow], showCol=["trace"])
        if narrowHover:
            trfig.update_layout({"width":220,"height":200})
        return trfig,"ok "
    except:
        return baseFig,"ERROR from plotHovered_callback: "+exc_info().__repr__() 

In [100]:
@app.callback(
    Output("selected_indices", "value"),
    [Input("roi_selector", "selectedData")]
)
def collectSelected(selData):
    try:
        out = np.unique([pt["hovertext"] for pt in selData["points"]])
        out = sorted(out,key=int)
        return ", ".join(out)
    except:
        return "" 

In [101]:
@app.callback(
    [Output("show_selected_traces", "figure"),
     Output("spare_out", "children"),
    ],
    [Input("selected_indices", "value")],
    [State("show_freq_input", "value"),
#      State("show_filtered_choice", "value"),
#      State("offset_input", "value"),
    ],
)
def plotTraces_callback(chosen, showFreq,
                        showFiltered=[],
                        offset=0,
                        forceOffset=False
                       ):
    try:
        if chosen=="": return baseFig,"waiting"
        global regions
        showFreq = float(showFreq)
        filtered = bool(len(showFiltered))
        if forceOffset:
            offset = float(offset)
        else:
            offset = float(offset) if filtered else 0
        colToshow = parseCol(regions, showFreq, filtered)
        if chosen=="":
            return baseFig, "nothing yet"
        try:
            ix = np.array(chosen.strip(" ,").split(","))
            ix = ix.astype(int)
            trfig = plotTraces(regions,indices=ix,time=regions.showTime[colToshow], showCol=[colToshow], offset=offset)
            trfig.update_layout({"width":800,"height":200+50*np.log(4*len(ix))})
            return trfig,"ok: "+repr((chosen, showFreq, showFiltered, offset))
        except:
            return baseFig,"error in chosen parsing"+repr(chosen)
    except:
        return baseFig,"ERROR from plotTraces_callback: "+exc_info().__repr__() 

In [102]:
# app.layout = html.Div(children=APP_LAYOUT)
# app

In [103]:
@app.callback(
    Output("save_feedback",    "children"),
    [Input("save_button",      "n_clicks")],
    [State("save_file_append", "value"   )]
             )
def save_callback(n_clicks,filename):

    try:
        if n_clicks <= 0:
            return "saving can take a few seconds."
        else:
            global regions
#             outDir = ".".join(regions.movieFilename.split(".")[:-1]+["_analysis"])
            outDir = os.path.split(regions.movieFilename)[0]+"/"
            return saveRois(regions, outDir, filename, movie=regions.movie)
    except:
            return "ERROR: "+exc_info().__repr__()

In [104]:
app.layout = html.Div(children=APP_LAYOUT)
app._repr_html_()
link2app = "https://ctn.physiologie.meduniwien.ac.at"+app.get_app_root_url()
HTML(f'open the following link in a different tab (do not close this tab!): <a href="{link2app}">{link2app}</a>')

In [53]:
exampleNpzFile

'local_data/Sandra/2020_03_06_Cosmin/Untitled_0_TEA10mM_20Hz_25ms_exposure_(30_sec)/TEA10mM_20Hz_25ms_exposure_(30_sec).tif'

In [36]:
# videoFilename = ".".join(regions.movieFilename.split(".")[:-1]+["mp4"])

# HTML(f"""<video width="400" height="222" controls="controls">
#   <source src="{videoFilename}" type="video/mp4" />
# </video>""")